# Imports & Setup

In [204]:
import requests
import json
import seaborn as sns
import plotly.express as px

# Standard Data Science Helpers
import numpy as np
import pandas as pd
import scipy

import plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
#init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(colorscale='plotly', world_readable=True)

# Extra options
pd.options.display.max_rows = 30
pd.options.display.max_columns = 25

# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import os
from IPython.display import Image, display, HTML

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import cv2
import matplotlib.pyplot as plt
import datetime as dt

%run login.py

loginquery = f"""
mutation {{
  logIn(
      email:\"{login}\",
      password:\"{pwd}\") {{
    jwt {{
      token
      exp
    }}
  }}
}}
"""

url = 'https://api.numina.co/graphql'
mylogin = requests.post(url, json={'query': loginquery})
token = mylogin.json()['data']['logIn']['jwt']['token']

devices = {'Streetscape': ('SWLSANDBOX1', 'img/Streetscape.png'), 'Outside': ('SWLSANDBOX3', 'img/Outside.png'),
           'UnderRaincoat': ('SWLSANDBOX2', 'img/UnderRaincoat.png')}

events = {'City Moments': (dt.datetime(2019, 8, 16, 19), dt.datetime(2019, 8, 17, 2)),
         'Open Sidewalk #4': (dt.datetime(2019, 3, 2, 15), dt.datetime(2019, 3, 2, 19)),
         'Sidewalk Summer Open House': (dt.datetime(2019, 6, 29, 12), dt.datetime(2019, 6, 29, 18)),
         'Startup Open House': (dt.datetime(2019, 9, 26, 16), dt.datetime(2019, 9, 26, 20)),
         'Tap:Ex Augmented Opera': (dt.datetime(2019, 11, 20, 18), dt.datetime(2019, 11, 23, 22))}

eventsDaySeperated = {'City Moments': (dt.datetime(2019, 8, 16, 19), dt.datetime(2019, 8, 17, 2)),
         'Open Sidewalk #4': (dt.datetime(2019, 3, 2, 15), dt.datetime(2019, 3, 2, 19)),
         'Sidewalk Summer Open House': (dt.datetime(2019, 6, 29, 12), dt.datetime(2019, 6, 29, 18)),
         'Startup Open House': (dt.datetime(2019, 9, 26, 16), dt.datetime(2019, 9, 26, 20)),
         'Tap:Ex Augmented Opera Day 1': (dt.datetime(2019, 11, 20, 18), dt.datetime(2019, 11, 20, 22)),
         'Tap:Ex Augmented Opera Day 2': (dt.datetime(2019, 11, 21, 18), dt.datetime(2019, 11, 21, 22)),
         'Tap:Ex Augmented Opera Day 3': (dt.datetime(2019, 11, 22, 18), dt.datetime(2019, 11, 22, 22)),
         'Tap:Ex Augmented Opera Day 4': (dt.datetime(2019, 11, 23, 18), dt.datetime(2019, 11, 23, 22))}

selectedEvent = events['City Moments']

OutsideZones = {'Tables': '[[245,0],[288,1],[468,0],[521,37],[559,81],[392,86],[307,84],[281,50]]',
               'Hedge': '[[448,2],[549,0],[640,0],[640,26],[638,60],[594,64],[544,66],[517,36]]',
               'Raincoat': '[[3,0],[104,0],[195,0],[195,24],[212,145],[168,153],[114,168],[0,209]]',
               'Tiles': '[[2,212],[89,179],[207,148],[220,207],[229,249],[255,362],[293,479],[0,479]]',
               'Concrete': '[[195,54],[267,31],[319,92],[638,78],[638,417],[571,460],[517,378],[292,469]]',
               'Leaf': '[[301,464],[380,430],[483,394],[506,384],[541,405],[561,478],[317,480],[309,478]]'}

RaincoatZones = {'Entrance': '[[263,0],[295,0],[326,0],[376,0],[533,0],[474,201],[112,122],[104,106]]',
                'Top Raincoat': '[[83,131],[157,137],[223,146],[299,157],[470,198],[454,313],[33,210],[25,194]]',
                'Right Raincoat': '[[217,256],[247,262],[298,272],[367,286],[460,307],[426,479],[42,480],[189,253]]',
                'Left Raincoat': '[[60,220],[127,233],[164,243],[186,249],[182,263],[90,479],[0,480],[0,212]]',
                'Right Concrete': '[[470,2],[636,1],[640,212],[638,252],[640,478],[516,477],[426,478],[478,186]]',
                'Left Concrete': '[[86,0],[252,0],[258,0],[210,29],[161,63],[108,112],[1,220],[1,0]]'}

StreetscapeZones = {'Back Building': '[[225,0],[269,0],[314,33],[280,54],[205,75],[135,99],[107,85],[71,35]]',
                    'Side Building': '[[403,0],[640,16],[640,59],[640,93],[640,113],[640,129],[495,81],[367,22]]',
                    'Tables': '[[314,41],[428,56],[502,89],[489,102],[440,136],[322,194],[229,123],[186,80]]',
                    'Left Walkway': '[[85,76],[181,185],[287,295],[399,410],[365,476],[201,480],[101,271],[45,91]]',
                    'Right Walkway': '[[640,159],[640,319],[531,480],[451,480],[366,470],[385,436],[433,386],[475,343]]',
                    'Stairs': '[[391,218],[438,258],[483,287],[506,308],[497,326],[448,362],[375,282],[323,241]]',
                    'Left of Tables': '[[188,72],[229,120],[283,156],[303,162],[321,180],[265,207],[172,136],[129,93]]',
                    'Right of Tables': '[[514,103],[569,124],[635,153],[587,210],[551,254],[503,293],[416,230],[353,195]]'}

In [179]:
OutsideZones

{'Tables': '[[245,0],[288,1],[468,0],[521,37],[559,81],[392,86],[307,84],[281,50]]',
 'Hedge': '[[448,2],[549,0],[640,0],[640,26],[638,60],[594,64],[544,66],[517,36]]',
 'Raincoat': '[[3,0],[104,0],[195,0],[195,24],[212,145],[168,153],[114,168],[0,209]]',
 'Tiles': '[[2,212],[89,179],[207,148],[220,207],[229,249],[255,362],[293,479],[0,479]]',
 'Concrete': '[[195,54],[267,31],[319,92],[638,78],[638,417],[571,460],[517,378],[292,469]]',
 'Leaf': '[[301,464],[380,430],[483,394],[506,384],[541,405],[561,478],[317,480],[309,478]]'}

In [123]:
for zone in OutsideZones.keys():
    OutsideZoneQuery = """
    mutation {
      createBehaviorZone(
        demarcation: %s, serialno: "SWLSANDBOX3", text: "%s"
      ) {
        behaviorZone {
          rawId
          demarcation
          text
        }
      }
    }
    """ % (OutsideZones[zone], zone)
    
    response = requests.post(url, json={'query': OutsideZoneQuery}, headers = {'Authorization':token}).json()
    OutsideZones[zone] = response['data']['createBehaviorZone']['behaviorZone']['rawId']

for zone in StreetscapeZones.keys():
    StreetscapeZoneQuery = """
    mutation {
      createBehaviorZone(
        demarcation: %s, serialno: "SWLSANDBOX1", text: "%s"
      ) {
        behaviorZone {
          rawId
          demarcation
          text
        }
      }
    }
    """ % (StreetscapeZones[zone], zone)
    
    response = requests.post(url, json={'query': StreetscapeZoneQuery}, headers = {'Authorization':token}).json()
    StreetscapeZones[zone] = response['data']['createBehaviorZone']['behaviorZone']['rawId']

for zone in RaincoatZones.keys():
    RaincoatZoneQuery = """
    mutation {
      createBehaviorZone(
        demarcation: %s, serialno: "SWLSANDBOX2", text: "%s"
      ) {
        behaviorZone {
          rawId
          demarcation
          text
        }
      }
    }
    """ % (RaincoatZones[zone], zone)
    
    response = requests.post(url, json={'query': RaincoatZoneQuery}, headers = {'Authorization':token}).json()
    RaincoatZones[zone] = response['data']['createBehaviorZone']['behaviorZone']['rawId']

In [187]:
eventDwellTimes = pd.DataFrame(columns=['event', 'serialno', 'time', 'mean', 'count', 'pct100', 'pct75', 'pct50', 'pct25'])

for event in eventsDaySeperated.keys():
    dwellTimesQueries = """
    query {
      feedDwellTimeDistribution(
        serialnos:["SWLSANDBOX1", "SWLSANDBOX2", "SWLSANDBOX3"],
        startTime:"%s",
        endTime:"%s",
        objClasses:["pedestrian"],
        interval:"1h") {
        edges {
          node {
            mean
            count
            pct100
            pct75
            pct50
            pct25
            serialno
            time
          }
        }
      }
    }
    """ % (eventsDaySeperated[event][0].strftime("%Y-%m-%dT%H:00:00"), eventsDaySeperated[event][1].strftime("%Y-%m-%dT%H:00:00"))

    response = requests.post(url, json={'query': dwellTimesQueries}, headers = {'Authorization':token}).json()

    for edge in response['data']['feedDwellTimeDistribution']['edges']:
        node = edge['node']
        node_row = pd.DataFrame([[event, node['serialno'], node['time'], node['mean'], node['count'], node['pct100'], node['pct75'], node['pct50'], node['pct25']]],
                    columns=['event', 'serialno', 'time', 'mean', 'count', 'pct100', 'pct75', 'pct50', 'pct25'])
        eventDwellTimes = eventDwellTimes.append(node_row)

eventDwellTimes = eventDwellTimes.reset_index(drop=True)
eventDwellTimes['count'] = eventDwellTimes['count'].astype('int64')
eventDwellTimes['mean'] = eventDwellTimes['mean'].fillna(0)
eventDwellTimes['pct100'] = eventDwellTimes['pct100'].fillna(0)
eventDwellTimes['pct75'] = eventDwellTimes['pct75'].fillna(0)
eventDwellTimes['pct50'] = eventDwellTimes['pct50'].fillna(0)
eventDwellTimes['pct25'] = eventDwellTimes['pct25'].fillna(0)

In [221]:
def eventsDwellTimesGraph(event, parameter):
    fig = px.line(eventDwellTimes[eventDwellTimes['event'] == event], x="time", y=parameter, color='serialno')
    fig.show()

interact(eventsDwellTimesGraph, event = list(eventsDaySeperated.keys()), parameter = ['mean', 'count', 'pct100', 'pct75', 'pct50', 'pct25'])

interactive(children=(Dropdown(description='event', options=('City Moments', 'Open Sidewalk #4', 'Sidewalk Sum…

<function __main__.eventsDwellTimesGraph(event, parameter)>

# Global Event Selector

In [171]:
def eventSelector(event):
    global selectedEvent
    if event == 'None':
        selectedEvent = 'None'
    else:
        selectedEvent = events[event]

eventNames = list(events.keys())
eventNames.append('None')

interact_manual(eventSelector, event=widgets.Dropdown(options=eventNames))


interactive(children=(Dropdown(description='event', options=('City Moments', 'Open Sidewalk #4', 'Sidewalk Sum…

<function __main__.eventSelector(event)>

# Dwell Time Table For Events

In [112]:
eventCounts = pd.DataFrame(columns=['event', 'count'])

for event in events.keys():
    startdate = events[event][0]
    enddate = events[event][1]
    
    eventCountsQuery = """
    query {
      feedCountMetrics(
        serialnos:["SWLSANDBOX1", "SWLSANDBOX2", "SWLSANDBOX3"],
        startTime:"%s",
        endTime:"%s",
        objClasses:["pedestrian"],
        timezone:"America/New_York",
        interval:"24h") {
        edges {
          node {
            serialno
            result
            objClass
            time
          }
        }
      }
    }
    """ % (startdate.strftime("%Y-%m-%dT%H:00:00"), enddate.strftime("%Y-%m-%dT%H:00:00"))
    
    response = requests.post(url, json={'query': eventCountsQuery}, headers = {'Authorization':token}).json()

    count = 0
    for edge in response['data']['feedCountMetrics']['edges']:
        count = count + edge['node']['result']
    
    this_event = pd.DataFrame([[event, count]], columns=['event', 'count'])
    eventCounts = eventCounts.append(this_event)
    
eventCounts = eventCounts.reset_index(drop=True)
eventCounts
    

,event,count
0,City Moments,257.0
1,Open Sidewalk #4,4975.0
2,Sidewalk Summer Open House,19533.0
3,Startup Open House,3748.0
4,Tap:Ex Augmented Opera,8556.0


# Dwell Time Comparison For Events

In [148]:
def dwellTimeEventsBar(startdate, enddate):
    
    dwellTimeEventsQuery = """
    query {
      feedCountMetrics(
        serialnos:["SWLSANDBOX1", "SWLSANDBOX2", "SWLSANDBOX3"],
        startTime:"%s",
        endTime:"%s",
        objClasses:["pedestrian"],
        timezone:"America/New_York",
        interval:"24h") {
        edges {
          node {
            serialno
            result
            objClass
            time
          }
        }
      }
    }
    """ % (startdate.strftime("%Y-%m-%dT00:00:00"), enddate.strftime("%Y-%m-%dT00:00:00"))    
    
    response = requests.post(url, json={'query': dwellTimeEventsQuery}, headers = {'Authorization':token}).json()

    count = 0
    for edge in response['data']['feedCountMetrics']['edges']:
        count = count + edge['node']['result']
        
    this_date = pd.DataFrame([['Selected Timeframe', count]], columns=['event', 'count'])
    
    eventCounts_copy = eventCounts
    eventCounts_copy = eventCounts_copy.append(this_date)
    
    plt.barh(eventCounts_copy['event'], eventCounts_copy['count'])

interact_manual(dwellTimeEventsBar, startdate=widgets.DatePicker(value=pd.to_datetime('2019-01-01')), 
                enddate=widgets.DatePicker(value=pd.to_datetime('2020-01-01')))

interactive(children=(DatePicker(value=Timestamp('2019-01-01 00:00:00'), description='startdate'), DatePicker(…

<function __main__.dwellTimeEventsBar(startdate, enddate)>

In [98]:
def getHeatmapImage(device, startdate, enddate):
    serial = devices[device][0]
    
    query = """
    query {
      feedHeatmaps(
        serialno: "%s",
        startTime:"%s",
        endTime:"%s",
        objClasses:["pedestrian"],
        timezone:"America/New_York") {
        edges {
          node {
            time
            objClass
            heatmap
          }
        }
      }
    }
    """ % (serial, startdate.strftime("%Y-%m-%dT%H:00:00"), enddate.strftime("%Y-%m-%dT%H:00:00"))

    heatdata = requests.post(url, json={'query': query}, headers = {'Authorization':token})
    
    heatarr = np.zeros((480,640))
    
    for point in heatdata.json()['data']['feedHeatmaps']['edges'][0]['node']['heatmap']:
        heatarr[point[1]][point[0]] = point[2]
    
    
    img = cv2.imread(devices[device][1])
    heatmap = cv2.resize(heatarr, (img.shape[1], img.shape[0]))
    heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    combined_image = cv2.addWeighted(img,1,heatmap,0.3,0)
    
    combined_image = cv2.cvtColor(combined_image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize = (15, 10))
    plt.axis('off')
    plt.imshow(combined_image)

In [172]:
interact_manual(getHeatmapImage, device=widgets.Dropdown(options=list(devices.keys())), 
                    startdate=widgets.DatePicker(value=pd.to_datetime('2019-01-01')),
             enddate=widgets.DatePicker(value=pd.to_datetime('2020-01-01')))

interactive(children=(Dropdown(description='device', options=('Streetscape', 'Outside', 'UnderRaincoat'), valu…

<function __main__.getHeatmapImage(device, startdate, enddate)>

In [220]:
def HeatmapEventSlider(device, event, day, time, cumulative):
    serial = devices[device][0]
    
    if not cumulative:
        if event == 'None':
            start = day + dt.timedelta(hours=time)
            end = start + dt.timedelta(hours=1)
        else:
            start = eventsDaySeperated[event][0] + dt.timedelta(hours=time)
            end = start + dt.timedelta(hours=1)
    else:
        if event == 'None':
            start = day + dt.timedelta(hours=0)
            end = start + dt.timedelta(hours=time)
        else:
            start = eventsDaySeperated[event][0]
            end = start + dt.timedelta(hours=time)
    
    query = """
    query {
      feedHeatmaps(
        serialno: "%s",
        startTime:"%s",
        endTime:"%s",
        objClasses:["pedestrian"],
        timezone:"America/New_York") {
        edges {
          node {
            time
            objClass
            heatmap
          }
        }
      }
    }
    """ % (serial, start.strftime("%Y-%m-%dT%H:00:00"), end.strftime("%Y-%m-%dT%H:00:00"))
    
    heatdata = requests.post(url, json={'query': query}, headers = {'Authorization':token})
    
    heatarr = np.zeros((480,640))
    
    for point in heatdata.json()['data']['feedHeatmaps']['edges'][0]['node']['heatmap']:
        heatarr[point[1]][point[0]] = point[2]
    
    
    img = cv2.imread(devices[device][1])
    heatmap = cv2.resize(heatarr, (img.shape[1], img.shape[0]))
    heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    combined_image = cv2.addWeighted(img,1,heatmap,0.3,0)
    
    combined_image = cv2.cvtColor(combined_image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize = (15, 10))
    plt.axis('off')
    plt.imshow(combined_image)

keys = list(eventsDaySeperated.keys())
keys.append('None')

interact_manual(HeatmapEventSlider, device=widgets.Dropdown(options=list(devices.keys())), 
                event=widgets.Dropdown(options=keys), day=widgets.DatePicker(value=pd.to_datetime('2019-01-01')),
                time=(0, 7), cumulative=widgets.Checkbox())

interactive(children=(Dropdown(description='device', options=('Streetscape', 'Outside', 'UnderRaincoat'), valu…

<function __main__.HeatmapEventSlider(device, event, day, time, cumulative)>

In [213]:
list(eventsDaySeperated.keys())

['City Moments',
 'Open Sidewalk #4',
 'Sidewalk Summer Open House',
 'Startup Open House',
 'Tap:Ex Augmented Opera Day 1',
 'Tap:Ex Augmented Opera Day 2',
 'Tap:Ex Augmented Opera Day 3',
 'Tap:Ex Augmented Opera Day 4']

In [116]:
def HeatmapEventSlider(device, day, time, cumulative):
    serial = devices[device][0]
    
    if not cumulative:
        start = day + dt.timedelta(hours=time)
        end = start + dt.timedelta(hours=1)
    else:
        start = day
        end = start + dt.timedelta(hours=time)
    
    query = """
    query {
      feedHeatmaps(
        serialno: "%s",
        startTime:"%s",
        endTime:"%s",
        objClasses:["pedestrian"],
        timezone:"America/New_York") {
        edges {
          node {
            time
            objClass
            heatmap
          }
        }
      }
    }
    """ % (serial, start.strftime("%Y-%m-%dT%H:00:00"), end.strftime("%Y-%m-%dT%H:00:00"))
    
    heatdata = requests.post(url, json={'query': query}, headers = {'Authorization':token})
    
    heatarr = np.zeros((480,640))
    
    for point in heatdata.json()['data']['feedHeatmaps']['edges'][0]['node']['heatmap']:
        heatarr[point[1]][point[0]] = point[2]
    
    
    img = cv2.imread(devices[device][1])
    heatmap = cv2.resize(heatarr, (img.shape[1], img.shape[0]))
    heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    combined_image = cv2.addWeighted(img,1,heatmap,0.3,0)
    
    combined_image = cv2.cvtColor(combined_image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize = (15, 10))
    plt.axis('off')
    plt.imshow(combined_image)

interact_manual(HeatmapEventSlider, device=widgets.Dropdown(options=list(devices.keys())), 
                day=widgets.DatePicker(value=pd.to_datetime('2019-01-01')), time=(0, 24), cumulative=widgets.Checkbox())

interactive(children=(Dropdown(description='device', options=('Streetscape', 'Outside', 'UnderRaincoat'), valu…

<function __main__.HeatmapEventSlider(device, day, time, cumulative)>